From : https://docs.python.org/3.6/reference/compound_stmts.html

In [ ]:
import os
os.getpid()

In [ ]:
import inspect
import hybridcuda
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
cures = hybridcuda.initcuda()
assert cures == 0

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

# 8. Compound statements


## 8.1 The `if` statement

https://docs.python.org/3.6/reference/compound_stmts.html#the-if-statement

## 8.2 The `while` statement

https://docs.python.org/3.6/reference/compound_stmts.html#the-while-statement

In [ ]:
@hybridfunction
def compound_while(a,b):
    x = 0
    while (x < a):
        b[0] = b[0] + 1
        x = x + 1

In [ ]:
b = [0]
a = 42

compound_while[1,1](a,b)
assert b == [42]

## 8.3 The `for` statement

https://docs.python.org/3.6/reference/compound_stmts.html#the-for-statement

## 8.6 Function definitions

https://docs.python.org/3.6/reference/compound_stmts.html#function-definitions

In [ ]:
def add(x,y):
    return x+y

@hybridfunction
def callfunc(a,b,c):
    c[0] = add(a,b)

In [ ]:
c = [0]

callfunc[1,1](30.0, 12, c)
assert c == [42.0]